In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium

df = pd.read_csv("raac_data.csv")

geolocator = Nominatim(user_agent="uk_schools_locator")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def geocode_address(address):
    try:
        location = geocode(f"{address}, UK")
        print(f"Geocoding successful for: {address}")  # Debug print
        return location.latitude, location.longitude
    except:
        print(f"Geocoding failed for: {address}")  # Debug print
        return None, None

# Apply geocoding
df[['latitude', 'longitude']] = df.apply(lambda row: geocode_address(row['Setting Name']), axis=1, result_type='expand')

# Drop rows where location wasn't found
df.dropna(subset=['latitude', 'longitude'], inplace=True)

# Print total number of rows that can be plotted
print(f"Total number of schools plotted on the map: {df.shape[0]}")

                                  Setting Name Publication Group  \
0     Atherton St George's CofE Primary School        16/10/2023   
1               Bushey and Oxhey Infant School        16/10/2023   
2  Camborne College (part of Cornwall College)        16/10/2023   
3                     Cann Hall Primary School        16/10/2023   
4                  Cheddington Combined School        16/10/2023   

        Setting Mitigation (16/10/2023) Setting Mitigation (14/09/2023) [1]  \
0  All pupils in face-to-face education                                 NaN   
1  All pupils in face-to-face education                                 NaN   
2  All pupils in face-to-face education                                 NaN   
3  All pupils in face-to-face education                                 NaN   
4  All pupils in face-to-face education                                 NaN   

  Setting Mitigation (30/08/2023) Setting Phase      Type of Setting  \
0                             NaN       Prim

Geocoding successful for: Donnington Wood Infant School and Nursery Centre
Geocoding successful for: East Bergholt High School
Geocoding successful for: East Tilbury Primary School
Geocoding successful for: Elmstead Primary School
Geocoding successful for: Eversley Primary School
Geocoding successful for: Ferryhill School
Geocoding successful for: Godinton Primary School
Geocoding successful for: Great Leighs Primary School
Geocoding successful for: Great Tey Church of England Voluntary Controlled Primary School
Geocoding successful for: Hadleigh High School
Geocoding successful for: Harlowbury Primary School
Geocoding successful for: Harwich and Dovercourt High School
Geocoding successful for: Hatfield Heath Primary School
Geocoding successful for: Hatfield Peverel St Andrew's Junior School
Geocoding failed for: Hatfield Peverel St Andrew's Junior School
Geocoding successful for: Henham and Ugley Primary and Nursery School
Geocoding successful for: Hillhouse CofE Primary School
Geocod

In [2]:
# Create a map object
map = folium.Map(location=[51.509865, -0.118092], zoom_start=6)

legend_html = '''
<div style="position: fixed; 
     top: 50px; right: 50px; width: 400px; height: 150px; 
     background-color: white; padding: 10px; 
     border:2px solid grey; z-index:9999; font-size:14px; overflow: auto;">
     &nbsp; <b>Color Code Legend</b> <br>
     &nbsp; Green: RAAC not present &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i><br>
     &nbsp; Red: Mix of face-to-face and remote arrangements &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i><br>
     &nbsp; Purple: All pupils in face-to-face education &nbsp; <i class="fa fa-map-marker fa-2x" style="color:purple"></i><br>
</div>
'''

# Apply the color determination function within the loop that creates markers
def determine_pin_color(row):
    # Check if "RAAC not present" is in the "Setting Mitigation (16/10/2023)" field or any other mitigation fields
    if row['Setting Mitigation (16/10/2023)'] == 'RAAC not present' or \
       row['Setting Mitigation (14/09/2023) [1]'] == 'RAAC not present' or \
       row['Setting Mitigation (30/08/2023)'] == 'RAAC not present':
        return 'green'
    # Check if "Mix of face-to-face and remote arrangements" is in the "Setting Mitigation (16/10/2023)" field
    elif row['Setting Mitigation (16/10/2023)'] == 'Mix of face-to-face and remote arrangements':
        return 'red'
    # Check if "All pupils in face-to-face education" is in the "Setting Mitigation (16/10/2023)" field
    elif row['Setting Mitigation (16/10/2023)'] == 'All pupils in face-to-face education':
        return 'purple'
    else:
        return 'blue'  # Default color

# Apply the color determination function within the loop that creates markers
for index, row in df.iterrows():
    # Construct the popup message with additional information
    popup_info = folium.Popup(
        (
            f"<strong>Setting Name:</strong> {row['Setting Name']}<br>"
            f"<strong>Setting Phase:</strong> {row['Setting Phase']}<br>"
            f"<strong>Type of Setting:</strong> {row['Type of Setting']}<br>"
            f"<strong>Setting Website:</strong> {row['Setting Website']}<br>"
            f"<strong>Local Authority:</strong> {row['Local Authority']}<br>"
            f"<strong>Parliamentary Constituency:</strong> {row['Parliamentary Constituency']}<br>"
            f"<strong>Region:</strong> {row['Region']}<br>"
            f"<strong>URN:</strong> {row['URN']}<br>"
            f"<strong>LAESTAB:</strong> {row['LAESTAB']}<br>"           
            f"<strong>Mitigation (30/08/2023):</strong> {row['Setting Mitigation (30/08/2023)']}<br>"
            f"<strong>Mitigation (14/09/2023):</strong> {row['Setting Mitigation (14/09/2023) [1]']}<br>"
            f"<strong>Mitigation (16/10/2023):</strong> {row['Setting Mitigation (16/10/2023)']}<br>"
        ),
        max_width=350
    )
    
    # Determine the color of the pin based on the mitigation status
    pin_color = determine_pin_color(row)
    
    # Add a marker to the map
    folium.Marker(
        [row['latitude'], row['longitude']],
        popup=popup_info,
        icon=folium.Icon(color=pin_color)  # This sets the color of the pin based on the mitigation status
    ).add_to(map)

# Add the legend to the map
map.get_root().html.add_child(folium.Element(legend_html))
# Display the map
map